<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/Test_Bonet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet

Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), done.


In [2]:
import Bonet

In [3]:
# -*- coding: utf-8 -*-

"""
Bone Age Assessment BoNet test routine.
"""

# Standard lib imports
import os
import csv
import glob
import time
import argparse
import warnings
import pandas as pd
import os.path as osp

In [4]:
# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
!pip install horovod
import horovod.torch as hvd
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.5 MB 37.3 MB/s 
  Created wheel for horovod: filename=horovod-0.26.1-cp38-cp38-linux_x86_64.whl size=28369888 sha256=08c79bcebbd00e695bc4f1181360d3938055c1faffed40398da146d5a6b70b2f
  Stored in directory: /root/.cache/pip/wheels/eb/0b/90/d53058f75f3ae3db9557f3e55dd8c016b2397e9b38557c8b66
Successfully built horovod


In [5]:
# Local imports
from Bonet.models.bonet import BoNet
from Bonet.data.data_loader import BoneageDataset 

In [6]:
# Other imports
from tqdm import tqdm
import pdb

In [7]:
warnings.filterwarnings("ignore")

parser = argparse.ArgumentParser()

In [8]:
# Dataloading-related settings
parser.add_argument('--heatmaps', default=False, action='store_true',
                help='Test model with gaussian heatmaps')
parser.add_argument('--cropped', default=False, action='store_true',
                help='Test model with cropped images according to bbox')
parser.add_argument('--dataset', default='RSNA', type=str,choices=['RSNA','RHPE'],
                help='Dataset to perform test')

_StoreAction(option_strings=['--dataset'], dest='dataset', nargs=None, const=None, default='RSNA', type=<class 'str'>, choices=['RSNA', 'RHPE'], help='Dataset to perform test', metavar=None)

In [9]:
# Dataloading-related settings
parser.add_argument('--data-test', default='data/test/', type=str,
                help='path to test data folder')
parser.add_argument('--ann-path-test', default='test.csv', type=str,
                help='path to BAA annotations file')
parser.add_argument('--rois-path-test', default='test.json',
                type=str, help='path to ROIs annotations in coco format')

parser.add_argument('--save-folder', default='TRAIN/new_test/',
                help='location to save checkpoint models')
parser.add_argument('--snapshot', default='boneage_bonet_weights.pth',
                help='path to weight snapshot file')


parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                help='number of data loading workers (default: 4)')

_StoreAction(option_strings=['-j', '--workers'], dest='workers', nargs=None, const=None, default=4, type=<class 'int'>, choices=None, help='number of data loading workers (default: 4)', metavar='N')

In [10]:
# Training procedure settings
parser.add_argument('--batch-size', default=1, type=int,
                help='Batch size for training')

parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--gpu', type=str, default='2,3')

_StoreAction(option_strings=['--gpu'], dest='gpu', nargs=None, const=None, default='2,3', type=<class 'str'>, choices=None, help=None, metavar=None)

In [11]:
args = parser.parse_args(args=[])

In [12]:
args_dict = vars(args)
print('Argument list to program')
print('\n'.join(['--{0} {1}'.format(arg, args_dict[arg])
                 for arg in args_dict]))
print('\n\n')

Argument list to program
--heatmaps False
--cropped False
--dataset RSNA
--data_test data/test/
--ann_path_test test.csv
--rois_path_test test.json
--save_folder TRAIN/new_test/
--snapshot boneage_bonet_weights.pth
--workers 4
--batch_size 1
--seed 1111
--gpu 2,3





In [13]:
torch.manual_seed(args.seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

In [14]:
if not os.path.exists(os.path.join(args.save_folder, 'inference')):
    os.makedirs(os.path.join(args.save_folder, 'inference'))

In [15]:
# Horovod settings
hvd.init()
torch.cuda.set_device(hvd.local_rank())
torch.cuda.manual_seed(hvd.size())

In [16]:
args.distributed = hvd.size() > 1
args.rank = hvd.rank()
args.size = hvd.size()

In [17]:
# CREATE THE NETWORK ARCHITECTURE AND LOAD THE BEST MODEL
if args.heatmaps:
    from Bonet.models.bonet_heatmap import BoNet
else:
    from Bonet.models.bonet import BoNet


In [18]:
net = BoNet()

In [19]:
if args.rank == 0:
    print('---> Number of params: {}'.format(
        sum([p.data.nelement() for p in net.parameters()])))

---> Number of params: 123172057


In [20]:
if osp.exists(args.snapshot):
    model_to_load=args.snapshot
else:
    model_to_load=args.save_folder+'/'+args.snapshot

In [21]:
if osp.exists(model_to_load) and args.rank == 0:
    print('Loading state dict from: {0}'.format(model_to_load))
    snapshot_dict = torch.load(model_to_load, map_location=lambda storage, loc: storage)
    weights= net.state_dict()
    new_snapshot_dict=snapshot_dict.copy()
    for key in snapshot_dict:
        if key not in weights.keys():
            new_key='inception_v3.'+key
            new_snapshot_dict[new_key]=snapshot_dict[key]
            new_snapshot_dict.pop(key)

    net.load_state_dict(new_snapshot_dict)

In [22]:
net = net.to(device)

In [23]:
# Criterion
criterion = nn.L1Loss()

In [24]:
# Horovod
hvd.broadcast_parameters(net.state_dict(), root_rank=0)

In [25]:
# Dataloader
test_transform = transforms.Compose([transforms.Resize((500, 500)),
                               transforms.ToTensor()])

In [26]:
if args.heatmaps:
    from Bonet.data.data_loader import Boneage_HeatmapDataset as Dataset
else:
    from Bonet.data.data_loader import BoneageDataset as Dataset


In [27]:
test_dataset = Dataset(args.data_test, args.ann_path_test,args.rois_path_test,
                                  img_transform=test_transform,crop=args.cropped,dataset=args.dataset)

FileNotFoundError: ignored